<a href="https://colab.research.google.com/github/RiccardoPrestigiacomo-99/example/blob/main/Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Classification Network (using Resnet32) on CIFAR100

In [22]:
import torch 
import torchvision
import torchvision.transforms as transforms

### Data Load

In [23]:
# we build a transform to normalize images: Data normalization is an important step which ensures 
# each input parameter (pixel, in this case) has a similar data distribution. This makes convergence 
# faster while training the network.
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
batch_size = 4

trainset = torchvision.datasets.CIFAR100(root='./data', train=True, 
                                         download=True, transform=transform)
# DataLoader. Combines a dataset and a sampler, and provides an iterable over the given dataset.
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR100(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)


Extracting ./data/cifar-100-python.tar.gz to ./data
Files already downloaded and verified


### Build the model

In [25]:
import torchvision.models as models

""" instantiate the net """
resnet34 = models.resnet18(pretrained=False)

""" move the model to the GPU """
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
resnet34.to(device)

### Define the loss and the optimization technique

In [27]:
import torch.optim as optim
import torch.nn as nn

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(resnet34.parameters(), lr=0.001, momentum=0.9)

### Training


In [31]:
#train the network

for epoch in range(3):

  running_loss = 0.0
  for i, data in enumerate(trainloader, 0):

    # get the inputs; data is a list of  [input,labels]
    inputs, labels = data[0].to(device), data[1].to(device)

    optimizer.zero_grad()

    outputs = resnet34(inputs)
    loss = criterion(outputs,labels)
    loss.backward()
    optimizer.step()

    running_loss += loss.item()
    if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

[1,  2000] loss: 3.901
[1,  4000] loss: 3.838
[1,  6000] loss: 3.806
[1,  8000] loss: 3.754
[1, 10000] loss: 3.685
[1, 12000] loss: 3.668
[2,  2000] loss: 3.539
[2,  4000] loss: 3.516
[2,  6000] loss: 3.444
[2,  8000] loss: 3.439
[2, 10000] loss: 3.397
[2, 12000] loss: 3.374
[3,  2000] loss: 3.245
[3,  4000] loss: 3.232
[3,  6000] loss: 3.194
[3,  8000] loss: 3.184
[3, 10000] loss: 3.169
[3, 12000] loss: 3.138


### Performance evaluation

In [32]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        # calculate outputs by running images through the network
        outputs = resnet34(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))


Accuracy of the network on the 10000 test images: 23 %


In [21]:
# divided our dataset into sample of 10 classes each
# train the network on the first 10 classes
# evaluate the network on the first 10 classes
# train the network on the second 10 classes (adding 10 output layers)
# evaluate the network on the first 20 classes
iterations=10
test = [] #initialized here because we test over all the classes not only those one in which I train
for i in range(iterations):
  i = 0
  classes_current_iter = range(i*iterations, i*iterations+iterations)
  train_iter = [] 
  for i in range(len(trainset)):
    if(trainset[i][-1] in classes_current_iter):
      test.append(trainset[i]) 
      train_iter.append(trainset[i])

  train_loader = torch.utils.data.DataLoader(train_iter, shuffle = True, batch_size=batch_size, num_workers=2)
  valid_loader = torch.utils.data.DataLoader(test, shuffle = True, batch_size = batch_size, num_workers=2)
  
  # train loader contains at each iteration the new 10 classes used to evaluate the network, while valid loader contains all classes seen so far

  # the idea si  


KeyboardInterrupt: ignored